In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

from collections import defaultdict, OrderedDict, Counter
import copy
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from datetime import date, datetime
import os

Populating the interactive namespace from numpy and matplotlib


In [20]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
# Otherwise it can be downloadad from https://grouplens.org/datasets/movielens/
# and Dataset created by Dataset.load_from_file()
# See doc: https://surprise.readthedocs.io/en/stable/getting_started.html#load-custom
# dataset_name = 'ml-100k'
dataset_name = 'ml-1m'
data = Dataset.load_builtin(dataset_name)

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cv_info = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Dataset ml-1m could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8735  0.8752  0.8757  0.8728  0.8722  0.8739  0.0014  
MAE (testset)     0.6859  0.6871  0.6873  0.6852  0.6845  0.6860  0.0011  
Fit time          50.75   49.15   47.39   46.40   46.94   48.13   1.60    
Test time         2.70    2.31    2.76    2.43    2.51    2.54    0.17    


In [21]:
trainset_full = data.build_full_trainset()
uids_raw = [trainset_full.to_raw_uid(uid_inner) for uid_inner in trainset_full.all_users()]
iids_raw = [trainset_full.to_raw_iid(iid_inner) for iid_inner in trainset_full.all_items()]

In [52]:
len(trainset_full.all_users())

6040

In [53]:
len(trainset_full.all_items())

3706

In [55]:
len(list(trainset_full.all_ratings()))

1000209

In [33]:
sorted(iids_raw, key=lambda x: int(x))

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '110',
 '111',
 '112',
 '113',
 '114',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '140',
 '141',
 '142',
 '

In [56]:
from surprise import get_dataset_dir
import io  # needed because of weird encoding of u.item file

def read_item_names(dataset_name):
    if dataset_name == "ml-100k":
        return read_item_names_100k()
    elif dataset_name == "ml-1m":
        return read_item_names_1m()
    raise ValueError("Wrong dataset_name {}".format(dataset_name))

def read_item_names_100k():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """
    
    file_name = get_dataset_dir() + '/{0}/{0}/u.item'.format(dataset_name)
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

def read_item_names_1m():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """
    
    file_name = get_dataset_dir() + '/ml-1m/ml-1m/movies.dat'.format(dataset_name)
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('::')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

def get_names_iid_inner(iidr_to_name, trainset_full):
    iidi_to_name = {} 
    name_to_iidi = {}
    for iid_raw, name in iidr_to_name.items():
        try:
            iid_inner = trainset_full.to_inner_iid(iid_raw)

            iidi_to_name[iid_inner] = name
            name_to_iidi[name] = iid_inner
        except ValueError:
            continue
        
    return iidi_to_name, name_to_iidi
    

iidr_to_name, name_to_iidr = read_item_names(dataset_name)
iidi_to_name, name_to_iidi = get_names_iid_inner(iidr_to_name, trainset_full)

In [58]:
for name in iidi_to_name.values():
    print(name)

Toy Story (1995)
Jumanji (1995)
Grumpier Old Men (1995)
Waiting to Exhale (1995)
Father of the Bride Part II (1995)
Heat (1995)
Sabrina (1995)
Tom and Huck (1995)
Sudden Death (1995)
GoldenEye (1995)
American President, The (1995)
Dracula: Dead and Loving It (1995)
Balto (1995)
Nixon (1995)
Cutthroat Island (1995)
Casino (1995)
Sense and Sensibility (1995)
Four Rooms (1995)
Ace Ventura: When Nature Calls (1995)
Money Train (1995)
Get Shorty (1995)
Copycat (1995)
Assassins (1995)
Powder (1995)
Leaving Las Vegas (1995)
Othello (1995)
Now and Then (1995)
Persuasion (1995)
City of Lost Children, The (1995)
Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)
Dangerous Minds (1995)
Twelve Monkeys (1995)
Wings of Courage (1995)
Babe (1995)
Carrington (1995)
Dead Man Walking (1995)
Across the Sea of Time (1995)
It Takes Two (1995)
Clueless (1995)
Cry, the Beloved Country (1995)
Richard III (1995)
Dead Presidents (1995)
Restoration (1995)
Mortal Kombat (1995)
To Die For (1995)
How to Make an A

Razor's Edge, The (1984)
Reds (1981)
Return with Honor (1998)
Time of the Gypsies (Dom za vesanje) (1989)
Days of Heaven (1978)
Fire Within, The (Le Feu Follet) (1963)
Love Bewitched, A (El Amor Brujo) (1986)
Lady Eve, The (1941)
Sullivan's Travels (1942)
Palm Beach Story, The (1942)
Man Facing Southeast (Hombre Mirando al Sudeste) (1986)
Niagara (1953)
Gilda (1946)
South Pacific (1958)
Flashdance (1983)
Indochine (1992)
Dirty Dozen, The (1967)
Mike's Murder (1984)
Help! (1965)
Goldfinger (1964)
From Russia with Love (1963)
Dr. No (1962)
Blue Lagoon, The (1980)
Fistful of Dollars, A (1964)
Hard 8 (a.k.a. Sydney, a.k.a. Hard Eight) (1996)
Home Alone 2: Lost in New York (1992)
Penitentiary II (1982)
Someone to Watch Over Me (1987)
Fight Club (1999)
Beefcake (1999)
Story of Us, The (1999)
Fever Pitch (1997)
Joe the King (1999)
Julien Donkey-Boy (1999)
Omega Code, The (1999)
Straight Story, The (1999)
Bad Seed, The (1956)
Time Bandits (1981)
Man and a Woman, A (Un Homme et une Femme) (1966

In [59]:
uid_raw = np.random.choice(uids_raw)
iid_raw = np.random.choice(iids_raw)
print(iidr_to_name[iid_raw])
algo.predict(uid_raw, iid_raw)

Hamlet (2000)


Prediction(uid='1409', iid='3598', r_ui=None, est=4.125594605208854, details={'was_impossible': False})

In [60]:
all_ratings = list(trainset_full.all_ratings())
uid_inner, iid_inner, r =  all_ratings[np.random.choice(len(all_ratings))]

uid_raw = trainset_full.to_raw_uid(uid_inner)
iid_raw = trainset_full.to_raw_iid(iid_inner)

print(iidr_to_name[iid_raw])
print(algo.predict(uid_raw, iid_raw))
print(uid_raw, iid_raw, r)

Serpico (1973)
user: 4253       item: 3735       r_ui = None   est = 3.56   {'was_impossible': False}
4253 3735 4.0


In [61]:
algo.pu.shape

(6040, 100)

In [62]:
algo.qi.shape

(3673, 100)

In [69]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(algo.qi)

# iidi = name_to_iidi["Star Wars (1977)"]
iidi = name_to_iidi["Toy Story (1995)"]
# iidi = name_to_iidi["Star Wars: Episode IV - A New Hope (1977)"]
distances, similar_iids_inner = nbrs.kneighbors(algo.qi[iidi:iidi+1, :])

similar_names = [iidi_to_name[iid_inner] for iid_inner in similar_iids_inner.ravel()]

print("Most similar movies to {}:".format(similar_names[0]))
for name in similar_names:
    print(" ", name)

Most similar movies to Toy Story (1995):
  Toy Story (1995)
  Picnic at Hanging Rock (1975)
  Muppet Treasure Island (1996)
  Eden (1997)
  Mission, The (1986)
  North (1994)
  Three Caballeros, The (1945)
  Happy Weekend (1996)
  8 1/2 Women (1999)
  Zero Kelvin (Kjærlighetens kjøtere) (1995)


# TODO
- [x] predict movie rating
- [x] see movie names
- [ ] recommend n movies to a user
- [ ] recommend with clustering
- [ ] t-SNE for movies and visualise